# Task: create the Python program which allows to Insert/Update from the local files and Export to the local files. 

### Import libraries

In [1]:
import mysql.connector
import pandas as pd
from sqlalchemy import create_engine
import pymysql

### Establish connections

In [2]:
# Establishing a connection to the database
cnx = mysql.connector.connect(user='root',
                              password='',
                              host='localhost',
                              database='countries')

In [3]:
# Creating a cursor object
cursor = cnx.cursor()

In [4]:
query2 = ("INSERT INTO countries (name, area, population) VALUES ('Lithuania',65300, 2615335 )")
cursor.execute(query2)

### Commit changes

In [5]:
# Commiting the transaction
cnx.commit()

In [6]:
query = "select * from countries"
cursor.execute(query)

In [7]:
# Fetching the results
for row in cursor.fetchall():
    print(row)

(14, 'Lithuania', 65300, 2615335)


### Close connections

In [8]:
# Closing the cursor and connection
cursor.close()
cnx.close()

## Import/Update MySQL database from local files

### Establish connection to mysql

In [9]:
engine = create_engine("mysql+pymysql://" + "root" + ":" + "@" + "localhost" + "/" + "countries")
conn = engine.connect()

### Provide local file name

In [13]:
local_file = input("Please provide a file name: ")
table = input("Please provide a table name that you want to update: ")

Please provide a file name: Cities.xlsx
Please provide a table name that you want to update: cities


### Read excel file

In [11]:
if table == "countries" and local_file == "Countries.xlsx":
    countries = pd.read_excel(local_file)
    
    # Update MySQL database from excel file
    countries.to_sql(name="countries",con=engine,if_exists="append",index=False)

In [14]:
if table == "cities" and local_file == "Cities.xlsx":
    cities = pd.read_excel(local_file, converters={"area":float,"country":str,"external_id":str}) 
    
    with engine.begin() as con:
        query = ("SELECT * FROM countries")
        countries_db = pd.read_sql(query,con)
    
    # Merge tables on country name
    cities = cities.merge(countries_db,left_on="country",right_on="name")
    cities = cities.drop(columns=["country","area_y","population_y","name_y"])

    # Rename columns
    cities = cities.rename(columns={"name_x":"name", "area_x": "area", "population_x": "population", "id":"country_id", "external_id":"ext_id"})
    
    # Update MySQL database from excel file
    cities.to_sql(name="cities",con=engine,if_exists="append",index=False)

In [15]:
with engine.begin() as con:
    query = ("SELECT * FROM cities")
    cities_db = pd.read_sql(query,con)

In [16]:
cities_db

,id,name,area,population,country_id,ext_id
0,18,Riga,307.0,605802,15,000001
1,19,Talinn,159.3,426538,16,000002


In [17]:
countries_db

,id,name,area,population
0,14,Lithuania,65300,2615335
1,15,Latvia,64589,1827457
2,16,Estonia,45228,1326535


## Export MySQL data to local files

In [18]:
countries_db.to_csv("Countries.csv",index=False,encoding="utf8")
countries_db.to_excel("Countries_from_sql.xlsx",index=False)

In [19]:
cities_db.to_csv("Cities.csv",index=False,encoding="utf8")
cities_db.to_excel("Cities_from_sql.xlsx",index=False)